In [ ]:
# 앙상블(ensemble)학습

In [10]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [11]:
from sklearn.preprocessing import MinMaxScaler
cancer_scaled = MinMaxScaler().fit_transform(cancer.data)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(
    cancer_scaled, cancer.target, stratify= cancer.target, test_size = 0.2, random_state=2021
)

### 1. 앙상블 학습 - Hard voting
- 로지스틱 회귀
- 서포트 벡터 머신
- K 최근접 이웃

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [14]:
lrc = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [18]:
from sklearn.ensemble import VotingClassifier

In [32]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc),('KNN',knn)],voting='hard'
)

In [33]:
voc.fit(X_train, y_train)
voc.score(X_train, y_train)

0.9758241758241758

In [34]:
# 개별 분류기의 성능


In [35]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

(0.9824561403508771, 0.9824561403508771, 0.9824561403508771)

In [36]:
### 2. 앙상블 학습 - 소프트 보팅


In [37]:
# 메소드와 속성
print(dir(lrc))  # 양쪽에 __는 시스템과 관련/ object의 성질을 상속

['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol', 'verbose', 'warm_start']


In [38]:
lrc.predict_proba(X_test[:5])

array([[0.36348222, 0.63651778],
       [0.97162943, 0.02837057],
       [0.1869565 , 0.8130435 ],
       [0.02133462, 0.97866538],
       [0.0548391 , 0.9451609 ]])

In [39]:
lrc.predict(X_test[:5])

array([1, 0, 1, 1, 1])

In [40]:
lrc.predict(X_test[:5])

array([1, 0, 1, 1, 1])

In [41]:
svc.predict_proba(X_test[:5])

AttributeError: predict_proba is not available when  probability=False

In [29]:
svc.get_params()  # probability=False인데 true로 바꾸면 가능

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [66]:
voc2 =SVC(probability= True)
voc2.fit(X_train, y_train)
voc2.predict_proba(X_test[:5])

array([[4.56934297e-01, 5.43065703e-01],
       [9.99837473e-01, 1.62527350e-04],
       [1.45217105e-02, 9.85478289e-01],
       [3.04255135e-03, 9.96957449e-01],
       [4.39649379e-03, 9.95603506e-01]])

In [67]:


voc2 = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc),('KNN',knn)],voting='soft'
)

In [68]:
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test)

AttributeError: predict_proba is not available when  probability=False

In [69]:
voc2.predict_proba(X_test[:5])

AttributeError: predict_proba is not available when  probability=False

- GridSearchCV 적용

In [73]:
params = {
    'LRC__C' : [0.01,0.05,0.1, 0.2, 0.5],
    'SVC__C' : [5,10,20,50]
    
}


In [74]:
from sklearn.model_selection import GridSearchCV
grid_voc2 = GridSearchCV(voc2, param_grid= params, scoring= 'accuracy', cv = 3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_voting.py", line 309, in predict
    maj = 

{'LRC__C': 0.01, 'SVC__C': 5}

In [72]:
grid_voc2.best_estimator_.score(X_test, y_test)

AttributeError: predict_proba is not available when  probability=False

In [75]:
params = {
    'LRC__C' : [0.01,0.05,0.1, 0.2, 0.5],
    'SVC__C' : [5,10,20,50]
    
}
grid_voc2 = GridSearchCV(voc2, param_grid= params, scoring= 'accuracy', cv = 3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_voting.py", line 309, in predict
    maj = 

{'LRC__C': 0.01, 'SVC__C': 5}

In [76]:
grid_voc2.best_estimator_.score(X_test, y_test)

AttributeError: predict_proba is not available when  probability=False

### 3. Random Forest
- DecisionTree 100개
-

In [78]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state= 2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [79]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9736842105263158